In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import networkx as nx
import plotly.graph_objects as go
from IPython.display import Image
import io

mount goole colab drive

In [ ]:
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive


In [ ]:
# read our data
df = pd.read_csv(r'E:\NZStudy\InternshipCS778\609365_1120054_bundle_archive\2020-03-29 Coronavirus Tweets.CSV')
df.head()

,status_id,user_id,created_at,screen_name,text,source,reply_to_status_id,reply_to_user_id,reply_to_screen_name,is_quote,...,retweet_count,country_code,place_full_name,place_type,followers_count,friends_count,account_lang,account_created_at,verified,lang
0,1244051646071611394,860252856829587457,2020-03-29T00:00:00Z,IMSS_SanLuis,"Ante cualquier enfermedad respiratoria, no te ...",TweetDeck,NaN,NaN,NaN,False,...,0,NaN,NaN,NaN,1008,41,NaN,2017-05-04T22:00:38Z,False,es
1,1244051645039706112,1125933654943895553,2020-03-29T00:00:00Z,intrac_ccs,#ATENCIÓN En el Terminal Nuevo Circo se implem...,TweetDeck,NaN,NaN,NaN,False,...,1,NaN,NaN,NaN,90,316,NaN,2019-05-08T01:21:16Z,False,es
2,1244051645975191557,80943559,2020-03-29T00:00:00Z,rlieving,“People are just storing up. They are staying ...,TweetDeck,NaN,NaN,NaN,False,...,0,NaN,NaN,NaN,136,457,NaN,2009-10-08T21:06:08Z,False,en
3,1244051646750928897,817072420947247104,2020-03-29T00:00:00Z,Tu_IMSS_Coah,"Si empezaste a trabajar, necesitas dar de alta...",TweetDeck,NaN,NaN,NaN,False,...,0,NaN,NaN,NaN,1549,170,NaN,2017-01-05T18:17:00Z,False,es
4,1244051647032102914,788863557349670913,2020-03-29T00:00:00Z,Tabasco_IMSS,Una sociedad informada está mejor preparada an...,TweetDeck,NaN,NaN,NaN,False,...,0,NaN,NaN,NaN,868,125,NaN,2016-10-19T22:05:03Z,False,es


# data cleaning

In [ ]:
# checking whether the status_id is unique or not
#df['status_id'].count()
df['status_id'].nunique()

In [ ]:
#drop duplicate
df2 = df.drop_duplicates(subset=['status_id'])
df2['status_id'].count()

564105

In [ ]:
df2['status_id'].is_unique

False

In [ ]:
df2.columns

Index(['status_id', 'user_id', 'created_at', 'screen_name', 'text', 'source',
       'reply_to_status_id', 'reply_to_user_id', 'reply_to_screen_name',
       'is_quote', 'is_retweet', 'favourites_count', 'retweet_count',
       'country_code', 'place_full_name', 'place_type', 'followers_count',
       'friends_count', 'account_lang', 'account_created_at', 'verified',
       'lang'],
      dtype='object')

In [ ]:
# dropping columns
tweet = df2.copy()
#tweet = tweet.drop(columns = ['user_id', 'source','account_lang', 'account_created_at'])
tweet = tweet.loc[:,['status_id', 'text','country_code','lang']]
tweet.head()

,status_id,text,country_code,lang
0,1244051646071611394,"Ante cualquier enfermedad respiratoria, no te ...",NaN,es
1,1244051645039706112,#ATENCIÓN En el Terminal Nuevo Circo se implem...,NaN,es
2,1244051645975191557,“People are just storing up. They are staying ...,NaN,en
3,1244051646750928897,"Si empezaste a trabajar, necesitas dar de alta...",NaN,es
4,1244051647032102914,Una sociedad informada está mejor preparada an...,NaN,es


In [ ]:
# filtering data with 'country_code = US' and 'language = en'
tweet_us = tweet.loc[(tweet.country_code =='US') & (tweet.lang == "en")].reset_index(drop = True)
tweet_us.drop(['country_code','lang'], axis=1, inplace = True)
tweet_us.head()

,status_id,text
0,1244051714933772293,@ClayTravis FLU? This seems fishy to me these ...
1,1244051767056191489,@netflix omg!!! Ozark ending was to die for!!!...
2,1244051810500988928,I don't think there is gd leadership across th...
3,1244051822354018304,Any other hospital doing this? Our engineering...
4,1244051953283448833,...We will do our best to make you smile with ...


In [ ]:
# get all the hash tag and put into a column
tweet_us['hash'] = tweet_us.text.str.findall(r'#.*?(?=\s|$)')

In [ ]:
tweet_us.head()

,status_id,text,hash
0,1244051714933772293,@ClayTravis FLU? This seems fishy to me these ...,[#Covid_19]
1,1244051767056191489,@netflix omg!!! Ozark ending was to die for!!!...,[#COVID19]
2,1244051810500988928,I don't think there is gd leadership across th...,[#COVID19]
3,1244051822354018304,Any other hospital doing this? Our engineering...,"[#Covid_19, #medtwitter]"
4,1244051953283448833,...We will do our best to make you smile with ...,"[#terotheotter, #covid_19, #lifeakua]"


In [ ]:
tweet_us.hash.isnull().sum()

0

In [ ]:
# hashtag list to multiple columns with various length
pd.DataFrame(tweet_us.hash.values.tolist(), tweet_us.index).add_prefix('hash_')

,hash_0,hash_1,hash_2,hash_3,hash_4,hash_5,hash_6,hash_7,hash_8,hash_9,...,hash_16,hash_17,hash_18,hash_19,hash_20,hash_21,hash_22,hash_23,hash_24,hash_25
0,#Covid_19,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,#COVID19,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,#COVID19,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,#Covid_19,#medtwitter,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,#terotheotter,#covid_19,#lifeakua,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7660,#coronavirus,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7661,#comfilife,#coccyx,#cushion,#sciatica,#pillow,#tonight,#SundayThoughts,#remoteworking,#weekthree,#Covid_19,...,None,None,None,None,None,None,None,None,None,None
7662,#gapol,#coronavirus,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7663,#socialdistancing,#coronavirus,#covid19,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
tweet_us.shape

(7665, 26)

In [ ]:
tweet_us.to_csv('tweet_us.csv')

# networkx

In [ ]:
# google colab file system
from google.colab import files
import io
uploaded = files.upload()

Saving tweet_us.csv to tweet_us.csv


In [ ]:
df = pd.read_csv(io.StringIO(uploaded['tweet_us.csv'].decode('utf-8')))
df.head()

,Unnamed: 0,code_0,code_1,code_2,code_3,code_4,code_5,code_6,code_7,code_8,code_9,code_10,code_11,code_12,code_13,code_14,code_15,code_16,code_17,code_18,code_19,code_20,code_21,code_22,code_23,code_24,code_25
0,0,#Covid_19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,#COVID19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,#COVID19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,#Covid_19,#medtwitter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,#terotheotter,#covid_19,#lifeakua,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# list of all hashtags
t=[]

#extracting all the hashtag from csv file
with open('tweet_us.csv','r') as f:
    mainlist=[]
    for line in f:
        temp=line.split(",")
        sublist=[]
        for word in temp:
            word=word.replace("'","")
            word=word.replace('"',"")
            word=word.lower()
            if word and word!='\n' and word.startswith("#"):
                sublist.append(word)
            if word and word.startswith("#"):
                t.append(word)
        mainlist.append(sublist)

In [ ]:
#get the hashtag list (control the hashtag list size)
hashtags=t[:1000].copy()
len(hashtags)

1000

In [ ]:
# control the data size for running
mainlist2= mainlist[0:1000]

In [ ]:
# An alternative way to get hashtag frequency
#values = {i:hashtags.count(i) for i in set(hashtags)}

In [ ]:
matrix=[[0]*len(hashtags)]*len(hashtags)
m=[]

final=[]

#dictionary of frequency of occurrence of each unique hashtag
values={}

new={}
for i,h1 in enumerate(hashtags):
  
    li=[]   
    for j,h2 in enumerate(hashtags): 
        c=0
        for sublist in mainlist2:
            if h1!=h2:
                if h2 in sublist and h1 in sublist:
                    c=c+1
    
        l=[]
      
        if c>10:        
            matrix[i][j]=c 
            final.append(h1)
            values[h1]=c
            l.append(h1)
            l.append(h2)
            l.append(c)
            tuple(l)
            li.append(h2)          
            m.append(l)

        j=j+1
    i=i+1
    if li:
        new[h1]=li

In [ ]:
len(t)

25110

In [ ]:
#values

In [ ]:
#nodes and their frequency
values.get('#covid_19')

22

In [ ]:
#intializing the networkx graph object
#G=nx.MultiGraph()
G=nx.Graph()

In [ ]:
#values.keys() contains list of all hashtags which are used as nodes here
G.add_nodes_from(values.keys())

In [ ]:
#adding edges
for k, v in new.items():
    G.add_edges_from(([(k, t) for t in v]))

In [ ]:
pos = nx.spring_layout(G)
for node in G.nodes:
    G.nodes[node]['pos'] = list(pos[node])

In [ ]:
node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=[],
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

In [ ]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

In [ ]:
for node, adjacencies in enumerate(G.adjacency()):
  node_trace['marker']['color']+=tuple([len(adjacencies[1])])
  node_info = str(adjacencies[0])
  node_trace['text']+=tuple([node_info])

In [ ]:
# size nodes by their frequency of occurrence
for i in values:
  node_trace['marker']['size']+=tuple([values[i]])

In [ ]:
#visualizing graph
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Network graph made with Python',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )


fig.show()

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
len(values2)

10

In [ ]:
text=[[0]*int(len(hashtags)/100)]*int(len(hashtags)/100)
text

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
new.items()

dict_items([('#covid_19', ['#covid19', '#covid19', '#coronavirus', '#coronavirus', '#covid19', '#coronalockdown', '#stayhome', '#coronavirus', '#covid19', '#covid19', '#quarantine', '#covid19', '#covid19', '#coronavirus', '#coronavirus', '#stayathomeandstaysafe', '#covid19', '#quarantine', '#socialdistancing', '#coronavirus', '#coronavirus', '#covid19', '#covid19', '#coronavirus', '#covid19', '#covid19', '#coronavirus', '#stayhome', '#covid19', '#covid19', '#covid19', '#covid19', '#coronavirus', '#coronavirus', '#coronavirus', '#covid19', '#covid19', '#coronavirus', '#covid19', '#quarantine', '#covid19', '#stayhome', '#covid19', '#coronavirus', '#covid19', '#covid19', '#quarantine', '#coronavirus', '#covid19', '#coronalockdown', '#coronavirus', '#coronavirus', '#coronavirus', '#coronavirus', '#coronavirus', '#covid19', '#covid19', '#covid19', '#covid19', '#coronavirus', '#stayathomeandstaysafe', '#stayathomeandstaysafe', '#coronalockdown', '#covid19', '#coronavirus', '#coronaupdate', '